In [64]:
import tensorflow.keras as keras
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from os import listdir

#Define path for training sets
PATHOLOGICAL_ID = "Pathological-Training/"
ACCEPTABLE_ID = "Non-Pathological-Training"

#Define functions to read files contained in training set
def pathological_path(filename):
    return os.path.join(PATHOLOGICAL_ID, filename)

def acceptable_path(filename):
    return os.path.join(ACCEPTABLE_ID, filename)

#Name of the folder for training sets
Names_Pathological = os.listdir('Pathological-Training')
Names_Acceptable = os.listdir('Non-Pathological-Training')

#Open first file and define pandas data frame for the pathological portion of the training set 
infile = open(pathological_path(Names_Pathological[1]),'r')
press_pathological = pd.read_csv(infile,sep = '\t',names = ('T','muB','P'))

infile = open(acceptable_path(Names_Acceptable[1]),'r')
press_acceptable = pd.read_csv(infile,sep = '\t',names = ('T','muB','P'))


#define tag for classification 0 = pathological choice, 1 = acceptable choice
rows = len(Names_Pathological)-2
cols = 1
tag0 = np.zeros((rows,cols))
tag0 = pd.DataFrame(tag0)

rows = len(Names_Acceptable)-1
cols = 1
tag1 = np.ones((rows,cols))
tag1 = pd.DataFrame(tag1)

#Import training files and define Pandas data frame

# for i in range(2,len(Names_Pathological)-2):
#     infile = open(pathological_path(Names_Pathological[i]),'r')
#     Press = pd.read_csv(infile,sep = '\t',names = ('T','muB','P'))
#     press_pathological =press_pathological.append(pd.DataFrame(Press))
    

# for i in range(2,len(Names_Acceptable)-1):
#     infile = open(acceptable_path(Names_Acceptable[i]),'r')
#     Press = pd.read_csv(infile,sep = '\t',names = ('T','muB','P'))
#     press_acceptable =press_acceptable.append(pd.DataFrame(Press))

#Now we need to reshape the data to matrices

#Define grid
Tgrid = 770
muBgrid = 451
elements = Tgrid*muBgrid
    
#Produce matrices


In [72]:
data = press_acceptable.iloc[0*elements:1*elements]['P']
x = len(data)

print(x/451)

# np.reshape(data,(Tgrid,muBgrid))


770.0


In [60]:
data.index.is_unique

True